In [ ]:
library(lmtp)

In [ ]:
set.seed(56)

n <- 1000
W <- rnorm(n, 10, 5)
A <- 23 + 5 * W + rnorm(n)
Y <- 7.2 * A + 3 * W + rnorm(n)

In [ ]:
ex1_dat <- data.frame(W, A, Y)
d <- function(data, x) data[[x]] - 5
psi1.1 <- lmtp_tmle(ex1_dat, "A", "Y", "W", shift = d,
                    outcome_type = "continuous", folds = 2)
psi1.1

In [ ]:
d <- function(data, x) (data[[x]] > 80) * (data[[x]] - 15) + (data[[x]] <= 80) * data[[x]]
psi1.2 <- lmtp_tmle(ex1_dat, "A", "Y", "W", shift = d,
                   outcome_type = "continuous", folds = 2)
psi1.2

In [ ]:
head(sim_t4)

In [ ]:
a <- c("A_1", "A_2", "A_3", "A_4")
tv <- list(c("L_1"), c("L_2"), c("L_3"), c("L_4"))
d <- function(data, trt) {
    a <- data[[trt]]
    (a - 1) * (a - 1 >= 1) + a * (a - 1 < 1)
}

progressr::with_progress({
    psi2.1 <- lmtp_tmle(sim_t4, a, "Y", time_vary = tv,
                        shift = d, folds = 2)
})
psi2.1

In [ ]:
progressr::with_progress({
    psi2.2 <- lmtp_tmle(sim_t4, a, "Y", time_vary = tv,
                       shift = d, folds = 2, k = 0)
})
psi2.2

In [ ]:
a <- c("A_1", "A_2", "A_3", "A_4")
time_varying <- list(c("L_1"), c("L_2"), c("L_3"), c("L_4"))
shift <- function(data, trt) {
    (data[[trt]] - 1) * (data[[trt]] - 1 >= 1) + data[[trt]] * (data[[trt]] - 1 < 1)
}
dynamic_mtp <- function(data, trt) {
    if (trt == "A_1") {
        shift(data, trt)
    } else {
        ifelse(data[[sub("A", "L", trt)]] == 1,
               shift(data, trt),
               data[[trt]])
    }
}
psi2.3 <- lmtp_tmle(sim_t4, a, "Y", time_vary = time_varying,
                    k = 0, shift = dynamic_mtp, folds = 2)
psi2.3

In [ ]:
for (i in a) {
    sim_t4[[i]] <- factor(sim_t4[[i]], levels = 0:5, ordered = TRUE)
}

d <- function(data, trt) {
    out <- list()
    a <- data[[trt]]
    for (i in 1:length(a)) {
        if (as.character(a[i]) %in% c("0", "1")) {
            out[[i]] <- as.character(a[i])
        } else {
            out[[i]] <- as.numeric(as.character(a[i])) - 1
        }
    }
    factor(unlist(out), levels = 0:5, ordered = TRUE)
}

progressr::with_progress({
    psi2.4 <- lmtp_tmle(sim_t4, a, "Y", time_vary = tv,
                        shift = d, k = 0, folds = 2)
})
psi2.4

In [ ]:
library(twang)
data(iptwExWide)

a <- paste0("tx", 1:3)
baseline <- c("gender", "age")
tv <- list(c("use0"), c("use1"), c("use2"))
progressr::with_progress({
    psi3.1 <- lmtp_tmle(iptwExWide, a, "outcome", baseline = baseline, time_vary = tv,
        shift = static_binary_on, outcome_type = "continuous", folds = 2)
})
psi3.1

In [ ]:
head(sim_cens)

In [ ]:
a <- c("A1", "A2")
tv <- list(c("L1"), c("L2"))
cens <- c("C1", "C2")
y <- "Y"
psi4.1 <- lmtp_tmle(sim_cens, a, y, time_vary = tv, cens = cens,
                    shift = NULL, folds = 2)
psi4.1

In [ ]:
d <- function(data, x) data[[x]] + 0.5
psi4.2 <- lmtp_tmle(sim_cens, a, y, time_vary = tv, cens = cens,
                    shift = d, folds = 2)
psi4.2

In [ ]:
a <- "trt"
y <- paste0("Y.", 1:6)
cens <- paste0("C.", 0:5)
baseline <- c("W1", "W2")
progressr::with_progress({
    psi5.1 <- lmtp_tmle(sim_point_surv, a, y, baseline, cens = cens,
                        shift = static_binary_on, folds = 2,
                        outcome_type = "survival")
})
psi5.1

In [ ]:
baseline <- c("W_1", "W_2")
trt <- c("A_1", "A_2")
time_vary <- list(c("L_11", "L_12"), 
                  c("L_21", "L_22"))
create_node_list(trt = trt, baseline = baseline, time_vary = time_vary, tau = 2)

In [ ]:
shift <- function(data, trt) {
  (data[[trt]] - 1) * (data[[trt]] - 1 >= 1) + data[[trt]] * (data[[trt]] - 1 < 1)
}

In [ ]:
A <- c("A_1", "A_2", "A_3", "A_4")
L <- list(c("L_1"), c("L_2"), c("L_3"), c("L_4"))
lmtp_sdr(sim_t4, A, "Y", time_vary = L, k = 0, shift = shift, folds = 5)